In [1]:
#import packages
import pandas as pd
import sys
import matplotlib.pyplot as plt
from datetime import datetime
import logging
import numpy as np
import copy

#custom classes
from classes import classes as cl
from functions import functions as fc

plt.rcParams.update({'font.size': 22})
logging.basicConfig(filename="rrp_sa_optimization.log", level=logging.INFO)

Initiation

In [2]:
fc.print_log("Starting initiation")
#create empty dict
initiation_objects = {'dict_depots': '', 'dict_sites': '', 'dict_plants': '', 'dict_jobs': '', 'dict_tours': '', 'list_days': ''}
init_path = 'output/inital_solution'

objects = fc.initiate(initiation_objects,init_path,'final')

#load objects
list_days = objects['list_days']
dict_depots = objects['dict_depots']
dict_sites = objects['dict_sites']
dict_plants = objects['dict_plants']
dict_jobs = objects['dict_jobs']
dict_tours = objects['dict_tours']

#also create lists for convenience
list_depots = [dict_depots[element] for element in dict_depots]
list_sites = [dict_sites[element] for element in dict_sites]
list_plants = [dict_plants[element] for element in dict_plants]
list_jobs = [dict_jobs[element] for element in dict_jobs]
list_tours  = [dict_tours[element] for element in dict_tours]


fc.print_log("Done with initiation")

Starting initiation at 22:45:15


In the first stage only one depot ist handled, so moves so its all about dropoff and pickup dates

In [3]:
#ov tours
for depot in dict_depots:
    total_tasks = 0
    total_distance = 0
    for day in list_days:
        total_tasks += dict_tours[depot][day].total_tasks
        total_distance += dict_tours[depot][day].distance
    print("Depot: {}, tasks: {} - distance: {}".format(depot,total_tasks,total_distance))



Depot: Neuherberg, tasks: 25454 - distance: 4547134.5148212565
Depot: Lauterhofen, tasks: 14329 - distance: 2151298.5445965845
Depot: Altbach, tasks: 5534 - distance: 648999.0083624776
Depot: Deisslingen, tasks: 0 - distance: 0
Depot: Bollschweil, tasks: 1586 - distance: 167723.71699792094
Depot: Lampertheim, tasks: 5872 - distance: 1059256.2914196497
Depot: Neuss, tasks: 19062 - distance: 2374315.453137417
Depot: Embsen, tasks: 9216 - distance: 1121736.368534145
Depot: Rottleberode, tasks: 25304 - distance: 6393335.900862844
Depot: Fenne, tasks: 0 - distance: 0
Depot: Castrop, tasks: 0 - distance: 0
Depot: Perl, tasks: 0 - distance: 0
Depot: Gnetsch, tasks: 0 - distance: 0
Depot: Vogelsdorf, tasks: 0 - distance: 0
Depot: Sukow, tasks: 0 - distance: 0


In [ ]:
fc.print_log("Start Inital Solution Plot")

#only work with one depot
temp_depot = dict_depots['Neuss']
list_depots = [temp_depot]
dict_depots = {'Neuss': temp_depot}



solution_initial = cl.Solution(temp_depot,dict_tours,list_days)

#figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

solution_initial.plot_tasks(50)
solution_initial.plot_task_proportion()
solution_initial.plot_distances(10000,1500)

fc.print_log("Done with  Inital Solution Plot")


After having a look at the inital solution, start optimizing

Stage 1: Try to level out the proportions between dropoff and pickup on eatch job

In [ ]:
fc.print_log("Start with Stage 1")

fc.print_log("Start deep copy")
dict_tours_temp = copy.deepcopy(dict_tours)
fc.print_log("Done with deep copy")
moves_possible = True
move_count = 0
iteration_count = 0


fc.print_log("Start Moves")
while moves_possible and iteration_count < 1000:
    move_made = False
    for depot in dict_depots:
        for day_org in list_days:

            #see if there are too many pickups in one job
            tour_org = dict_tours_temp[depot][day_org]

            if tour_org.total_pickups > tour_org.total_dropoffs:
                move_job = tour_org.worst_edge_pickup

                #see if there is space for a pickup, and check if it works with the earliest pickup date of the job
                for day_new in list_days:
                    tour_new = dict_tours_temp[depot][day_new]
                    if tour_new.total_pickups < tour_new.total_dropoffs\
                            and move_job.end > day_new:

                        #move job to new tour
                        tour_org.list_pickups.pop(move_job)
                        tour_new.list_pickups.append(move_job)

                        #adjust values in Tour class
                        tour_org.update_totals()
                        tour_new.update_totals()
                        tour_org.distance_uptodate = False
                        tour_new.distance_uptodate = False

                        #adjust values in Job class
                        move_job.pickup_day = day_new

                        #adjust move_bol and leave for loop
                        move_made = True
                        move_count +=1
                        break
    iteration_count += 1
    moves_possible = move_made

    print("Current: Iterations: {} - Moves: {}".format(move_count,iteration_count),end='\x1b[1K\r')

if(iteration_count > 999):
    fc.print_log("iteration exit")

fc.print_log("Final: Iterations: {} - Moves: {}".format(move_count,iteration_count))

fc.print_log("Done with Moves")
solution_stage1 = cl.Solution(temp_depot,dict_tours_temp,list_days)

solution_initial.plot_tasks(50)
solution_initial.plot_task_proportion()

solution_stage1.plot_tasks(50)
solution_stage1.plot_task_proportion()
fc.print_log("Done with Stage 1")